## Barrel shifter (cyhdl library)

This variant uses mixed myIRL and cythonized myhdl code segments. Note that the procedural generation is quite a bit faster.

We import the barrelshifter library:

In [1]:
from library.barrelshifter import *

Compiling /tmp/_cython_inline_2ce85e89b2692d4e9a260773f5d073fbdd3a2276.pyx because it changed.
[1/1] Cythonizing /tmp/_cython_inline_2ce85e89b2692d4e9a260773f5d073fbdd3a2276.pyx


[Edit barrelshifter.py](library/barrelshifter.py)

Then we create a barrel shifter generator with data width 32 (power 5):

In [2]:
W_POWER = 5

TEST_VALUES = [
    (0xdead, 8, 0xdead << 8),
    (0x8f01, 15, 0x8f01 << 15),
]

b = BarrelShifterGenerator(W_POWER)

Then we instance it first time in the test bench below. Note the instancing time on this first run.

In [5]:
from myirl.test.common_test import gen_osc
from cyhdl import *

@block
def top_bs(b):
    clk = ClkSignal()
    ce = Signal(bool())
    val, result = [ Signal(intbv(0xaa00)[2 ** W_POWER:]) for i in range(2) ]
    s = Signal(intbv()[W_POWER:])
    
    inst = [
        b.barrel_shifter(clk, ce, val, s, result, False),
        gen_osc(clk, 2)
    ]


    @instance
    def stim():
        for item in TEST_VALUES:
            ce.next = False
            s.next = item[1]
            val.next = item[0]
            yield(clk.posedge)
            ce.next = True
            yield(clk.posedge)
            yield(clk.posedge)

            print(result)
            assert result == item[2]
            
        raise StopSimulation

    
    inst += [ stim ]
    return inst
 
@utils.timer
def test(b):    
    return top_bs(b)

design = test(b)

Finished test in 0.0442 secs


Running the test bench again, the duration is a little less than on the first run, because this module was already compiled.

In [6]:
design = test(b)

 Module top_bs: Existing instance top_bs, rename to top_bs_1 
Finished test in 0.0527 secs


Finally, the quick & dirty verification:

In [7]:
from myirl import targets
from myirl.test.common_test import run_ghdl

f = design.elab(targets.VHDL, elab_all = True)
f += b.elab(targets.VHDL)
# print(f)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = True)

 Writing 'top_bs_1' to file /tmp/myirl_top_bs_92fvx14l/top_bs_1.vhdl 
 DEBUG: omit `b` from interface, (passthrough type <class '_cython_inline_2ce85e89b2692d4e9a260773f5d073fbdd3a2276.__invoke.<locals>.BarrelShifterGenerator'>) 
 Creating library file module_defs.vhdl 
 Writing 'shifter_stage_4' to file shifter_stage_4.vhdl 
DEBUG SKIP MODULE (LIB: BarrelShifterGenerator 'bs') <class '_cython_inline_2ce85e89b2692d4e9a260773f5d073fbdd3a2276.__invoke.<locals>.BarrelShifterGenerator'>
 DEBUG: omit `self` from interface, (passthrough type <class '_cython_inline_2ce85e89b2692d4e9a260773f5d073fbdd3a2276.__invoke.<locals>.BarrelShifterGenerator'>) 
 DEBUG: omit `nmux` from interface, (passthrough type <class 'int'>) 
 DEBUG: omit `asr` from interface, (passthrough type <class 'bool'>) 
 DEBUG: omit `rotate` from interface, (passthrough type <class 'bool'>) 
 Writing 'shifter_stage_3' to file shifter_stage_3.vhdl 
DEBUG SKIP MODULE (LIB: BarrelShifterGenerator 'bs') <class '_cython_inline_2ce

/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/kernel/components.py:211: UserWarning: @component `shifter_stage_4` interface :Unspecified port I/O 'msb' => IN
  base.warnings.warn((msg + " => IN") % n)
/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/kernel/components.py:211: UserWarning: @component `shifter_stage_4` interface :Unspecified port I/O 'sbit' => IN
  base.warnings.warn((msg + " => IN") % n)
/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/kernel/components.py:211: UserWarning: @component `shifter_stage_3` interface :Unspecified port I/O 'msb' => IN
  base.warnings.warn((msg + " => IN") % n)
/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/kernel/components.py:211: UserWarning: @component `shifter_stage_3` interface :Unspecified port I/O 'sbit' => IN
  base.warnings.warn((msg + " => IN") % n)
/home/testing/.loc

==== COSIM stdout ====
0x00DEAD00
0x47808000
simulation stopped @22ns



0

In [8]:
# !cat -n /tmp/barrel_shifter.vhdl